## 🔑 Lab #3-3 과제 정답지

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error
import plotly.express as px

In [ ]:
# RMSLE 계산을 위한 함수
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

In [ ]:
# 데이터 로드
path = '../../datasets/ml/bike-sharing/SeoulBikeData.csv'
df = pd.read_csv(path, encoding='cp949')


In [ ]:
# 데이터 전처리
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Hour'] = df['Hour']

df.drop(['Date', 'Seasons', 'Holiday', 'Functioning Day'], axis=1, inplace=True)

X = df.drop('Rented Bike Count', axis=1)
y = df['Rented Bike Count']

In [ ]:
# 음수 값을 0으로 변환 (로그 변환 시 오류 방지)
y[y < 0] = 0

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("데이터 준비 완료!")
X_train.head()

데이터 준비 완료!


/var/folders/5y/vwfpdbc96lb5kxd7pmfd9rl00000gn/T/ipykernel_83004/687112193.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[y < 0] = 0


,Hour,Temperature(캜),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(캜),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Year,Month,Day
8415,15,13.2,61,3.9,719,5.8,1.03,0.0,0.0,2018,11,16
5049,9,22.9,86,1.7,538,20.4,0.76,0.0,0.0,2018,6,29
8395,19,11.2,46,1.4,869,0.0,0.00,0.0,0.0,2018,11,15
1535,23,-2.6,69,2.0,1434,-7.5,0.00,0.0,0.0,2018,2,2
5518,22,27.2,73,1.5,1005,21.9,0.00,0.0,0.0,2018,7,18


In [ ]:
# 기본 모델 정의
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
xgb_model = XGBRegressor(n_estimators=100, random_state=42)
lgbm_model = LGBMRegressor(n_estimators=100, random_state=42)

In [ ]:
# 메타 모델 정의
meta_model = LinearRegression()

base_models = [rf_model, xgb_model, lgbm_model]

In [ ]:
# K-Fold 설정
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# OOF 예측값을 저장할 배열 초기화
oof_preds = np.zeros((X_train.shape[0], len(base_models)))
test_preds = np.zeros((X_test.shape[0], len(base_models)))

In [ ]:
print("OOF 예측 생성을 시작합니다...")
# 각 기본 모델에 대해 K-Fold 학습 및 예측 수행
for i, model in enumerate(base_models):
    print(f"모델 {model.__class__.__name__} 학습 중...")
    
    # K-Fold 반복
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train)):
        # Fold 데이터 분리
        X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        # 모델 학습
        model.fit(X_train_fold, y_train_fold)
        
        # OOF 예측값 생성
        oof_preds[val_idx, i] = model.predict(X_val_fold)
        
        # 테스트 데이터 예측값 생성
        test_preds[:, i] += model.predict(X_test)
        
        print(f"  - Fold {fold+1} 완료")

OOF 예측 생성을 시작합니다...
모델 RandomForestRegressor 학습 중...
  - Fold 1 완료
  - Fold 2 완료
  - Fold 3 완료
  - Fold 4 완료
  - Fold 5 완료
모델 XGBRegressor 학습 중...
  - Fold 1 완료
  - Fold 2 완료
  - Fold 3 완료
  - Fold 4 완료
  - Fold 5 완료
모델 LGBMRegressor 학습 중...
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001015 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1285
[LightGBM] [Info] Number of data points in the train set: 5606, number of used features: 12
[LightGBM] [Info] Start training from score 702.360507
  - Fold 1 완료
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000479 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory i

In [ ]:
# 테스트 데이터 예측값의 평균 계산
test_preds /= kf.n_splits

print("\nOOF 예측 생성 완료!")
print("OOF 예측 데이터 shape:", oof_preds.shape)
print("Test 예측 데이터 shape:", test_preds.shape)


OOF 예측 생성 완료!
OOF 예측 데이터 shape: (7008, 3)
Test 예측 데이터 shape: (1752, 3)


In [ ]:
# 메타 모델 학습
meta_model.fit(oof_preds, y_train)

# 최종 예측
final_predictions = meta_model.predict(test_preds)

# 음수 값 처리
final_predictions[final_predictions < 0] = 0

# RMSLE로 최종 성능 평가
final_rmsle = rmsle(y_test, final_predictions)
print(f"\n수동 구현 스태킹 모델의 최종 RMSLE: {final_rmsle:.4f}")


수동 구현 스태킹 모델의 최종 RMSLE: 1.0923


In [ ]:
# 결과 시각화
fig = px.scatter(
    x=y_test,
    y=final_predictions,
    labels={'x': '실제 대여량', 'y': '예측 대여량'},
    title='스태킹 모델 예측 결과: 실제값 vs. 예측값',
    opacity=0.5
)

fig.add_shape(
    type='line',
    x0=y_test.min(), y0=y_test.min(),
    x1=y_test.max(), y1=y_test.max(),
    line=dict(color='Red', width=2, dash='dash')
)

fig.update_layout(width=800, height=600)
fig.show()